In [5]:
import pandas as pd
import numpy as np
import os
import glob
import ast # 문자열 리스트 변환용
from datetime import datetime

# --- 0. Configuration ---
ACCIDENT_RAW_FILE = "./accident/전국 산악사고 구조활동현황(2017~2021).csv"
WEATHER_DATA_DIR = './weather/'
VISITOR_RAW_FILE = "./accident/국립공원공단_국립공원 시간별 일별 탐방객 통계_20221024.csv"
FINAL_OUTPUT_MERGED_FILE = "preprocessed_data.csv" # 파일명 변경
COMMON_DATE_COLUMN = 'Date'
START_DATE_STR = "2018-01-01"
END_DATE_STR = "2022-09-30"
TARGET_DISTRICT_FOR_VISITORS = "설악동"
TARGET_WEATHER_STATION_NAME = '속초' # 날씨 데이터를 필터링할 지점명

# Column Mapping Dictionaries (이전과 동일하게 유지 또는 필요시 수정)
weather_col_map_to_std_korean = {
    '지점번호': ['지점번호'], '지점명': ['지점명'], '일시': ['일시', '날짜'],
    '평균풍속(m/s)': ['평균풍속(m/s)', '평균풍속'], '최대풍속(m/s)': ['최대풍속(m/s)', '최대풍속'],
    '평균기온(℃)': ['평균기온(℃)', '평균기온'], '최고기온(℃)': ['최고기온(℃)', '최고기온'],
    '최저기온(℃)': ['최저기온(℃)', '최저기온'], '최고기온시각': ['최고기온시각'],
    '최저기온시각': ['최저기온시각'], '일교차': ['일교차'],
    '평균습도(%rh)': ['평균습도(%rh)', '평균습도'], '강수량(mm)': ['강수량(mm)', '강수량']
}
weather_col_map_korean_to_english = {
    '일시': COMMON_DATE_COLUMN, '최대풍속(m/s)': 'MaxWindSpd(m/s)', '평균풍속(m/s)': 'AvgWindSpd(m/s)',
    '최고기온(℃)': 'MaxTempC(℃)', '최저기온(℃)': 'MinTempC(℃)', '평균기온(℃)': 'AvgTempC(℃)',
    '최고기온시각': 'TimeOfMaxTempC', '최저기온시각': 'TimeOfMinTempC', '일교차': 'Diurnal_Temperature_Range',
    '평균습도(%rh)': 'Avg_Humidity_pct(%rh)', '강수량(mm)': 'Precipitation_mm(mm)'
}
accident_col_map_korean_to_english = {
    '신고년월일': COMMON_DATE_COLUMN, '발생장소_구': 'Accident_City_District',
    '구조인원': 'Total_Rescued_Count', '사고원인코드명_사고종별': 'Accident_Cause_Raw',
    '처리결과코드': 'Outcome_Code_Raw'
}
visitor_col_map_korean_to_english = {
    '일자': COMMON_DATE_COLUMN, '관리지구': 'District_Name',
    '전체 탐방객수': 'Total_Visitor_Count_Raw'
}

# --- Helper Functions ---
def try_read_csv_with_encodings(file_path):
    # ... (이전과 동일) ...
    for enc in ['utf-8', 'cp949', 'euc-kr', 'utf-8-sig']:
        try: return pd.read_csv(file_path, encoding=enc)
        except UnicodeDecodeError: continue
        except Exception as e: print(f"Error reading {file_path} with {enc}: {e}"); continue
    print(f"Warning: Could not read or decode {file_path}. Skipping this file.")
    return None

def normalize_and_rename_cols(df, std_korean_map, eng_map=None):
    # ... (이전과 동일, inplace=True 사용) ...
    df.columns = [col.strip() for col in df.columns]
    for target_col, possible_names in std_korean_map.items():
        for col_name in possible_names:
            if col_name in df.columns and target_col != col_name:
                df.rename(columns={col_name: target_col}, inplace=True)
                break
    if eng_map:
        rename_map_eng = {k: v for k, v in eng_map.items() if k in df.columns}
        if rename_map_eng: df.rename(columns=rename_map_eng, inplace=True)
    return df

def process_date_col(df, current_date_col_name, target_date_col_name):
    # ... (이전과 동일, errors='coerce' 후 dropna 고려) ...
    if current_date_col_name not in df.columns:
        print(f"Warning: Date column '{current_date_col_name}' not found. Skipping date processing."); return df
    df[current_date_col_name] = df[current_date_col_name].astype(str).str.replace('.', '-', regex=False).str.strip()
    df[target_date_col_name] = pd.to_datetime(df[current_date_col_name], errors='coerce')
    if current_date_col_name != target_date_col_name:
        df.drop(columns=[current_date_col_name], inplace=True, errors='ignore')
    df.dropna(subset=[target_date_col_name], inplace=True) # 유효하지 않은 날짜 행 제거
    return df

# --- Data Processing Functions for Each Source ---
def process_weather_data(weather_dir, std_korean_map, eng_map, date_col, target_station_name):
    print("\n--- 1. Processing Weather Data ---")
    csv_files = glob.glob(os.path.join(weather_dir, '*.csv'))
    csv_files = [f for f in csv_files if not (f.endswith('combined_weather_data.csv') or f.endswith(FINAL_OUTPUT_MERGED_FILE))]
    
    if not csv_files: print("No weather CSV files found."); return pd.DataFrame(columns=[date_col])

    # 기준이 될 전체 날짜 범위 생성 (병합의 기준으로 사용)
    try:
        start_dt = pd.to_datetime(START_DATE_STR); end_dt = pd.to_datetime(END_DATE_STR)
        df_weather_merged = pd.DataFrame(pd.date_range(start=start_dt, end=end_dt, freq='D'), columns=[date_col])
    except Exception as e: print(f"Error creating date range for weather: {e}"); return pd.DataFrame(columns=[date_col])

    for file in csv_files:
        print(f"  Processing weather file: {file}")
        df_w_component_raw = try_read_csv_with_encodings(file)
        if df_w_component_raw is None: continue
        
        df_w_component = normalize_and_rename_cols(df_w_component_raw, std_korean_map) # 1차: 표준 한글
        df_w_component = process_date_col(df_w_component, '일시', date_col) # 표준 한글 '일시' -> 최종 'Date'

        if '지점명' in df_w_component.columns and target_station_name:
            df_w_component = df_w_component[df_w_component['지점명'] == target_station_name].copy()
            df_w_component.drop(columns=['지점번호', '지점명'], inplace=True, errors='ignore')
        
        if not df_w_component.empty and date_col in df_w_component.columns:
            df_w_component = df_w_component.groupby(date_col, as_index=False).first() # 날짜별 유일값
            # 영문명 변경은 모든 컴포넌트 병합 후 한 번에 하거나, 여기서 각 컴포넌트별로 수행
            df_w_component = normalize_and_rename_cols(df_w_component, {}, eng_map) # 2차: 영문명 (eng_map만 사용)
            
            # COMMON_DATE_COLUMN 외의 컬럼만 가져와서 병합
            cols_to_merge = [date_col] + [col for col in df_w_component.columns if col != date_col and col not in df_weather_merged.columns]
            if len(cols_to_merge) > 1:
                 df_weather_merged = pd.merge(df_weather_merged, df_w_component[cols_to_merge], on=date_col, how='left')
    
    if df_weather_merged.empty or len(df_weather_merged.columns) <= 1:
        print("Warning: No valid weather data could be processed and merged."); 
        return pd.DataFrame(columns=[date_col]) # 빈 DataFrame 반환
    print(f"Weather data processed and merged. Shape: {df_weather_merged.shape}")
    return df_weather_merged

def process_accident_data(accident_file, eng_map, date_col, start_date, end_date):
    print("\n--- 2. Processing Accident Data ---")
    df_raw = try_read_csv_with_encodings(accident_file)
    if df_raw is None: return pd.DataFrame(columns=[date_col])
    
    df_processed = normalize_and_rename_cols(df_raw, {}, eng_map) # 바로 영문명으로
    df_processed = process_date_col(df_processed, date_col, date_col) # COMMON_DATE_COLUMN으로 이미 변경됨

    city_col = 'Accident_City_District' if 'Accident_City_District' in df_processed.columns else '발생장소_구'
    if city_col in df_processed.columns:
        df_sokcho = df_processed[df_processed[city_col] == '속초시'].copy()
    else: df_sokcho = df_processed.copy()

    if not df_sokcho.empty and date_col in df_sokcho.columns:
        # 날짜 범위 필터링
        df_sokcho = df_sokcho[(df_sokcho[date_col] >= pd.to_datetime(start_date)) & (df_sokcho[date_col] <= pd.to_datetime(end_date))]
        if df_sokcho.empty: print("No Sokcho accident data in the specified date range."); return pd.DataFrame(columns=[date_col, 'Total_Rescued_Count', 'Accident_Cause_List', 'Accident_Outcome_List'])

        def list_agg(series): return [str(item) for item in series if pd.notna(item) and str(item).strip() != '']
        agg_rules = {'Total_Rescued_Count': ('Total_Rescued_Count', 'sum')}
        if 'Accident_Cause_Raw' in df_sokcho.columns: agg_rules['Accident_Cause_List'] = ('Accident_Cause_Raw', list_agg)
        if 'Outcome_Code_Raw' in df_sokcho.columns: agg_rules['Accident_Outcome_List'] = ('Outcome_Code_Raw', list_agg)
        df_summary = df_sokcho.groupby(date_col, as_index=False).agg(**agg_rules)
        print(f"Sokcho accident data summarized. Shape: {df_summary.shape}")
        return df_summary
    print("No Sokcho accident data to summarize."); return pd.DataFrame(columns=[date_col, 'Total_Rescued_Count', 'Accident_Cause_List', 'Accident_Outcome_List'])

def process_visitor_data(visitor_file, eng_map, date_col, target_district, start_date, end_date):
    print("\n--- 3. Processing Visitor Data ---")
    df_raw = try_read_csv_with_encodings(visitor_file)
    if df_raw is None: return pd.DataFrame(columns=[date_col]), pd.DataFrame(columns=[date_col])

    df_processed = normalize_and_rename_cols(df_raw, {}, eng_map)
    df_processed = process_date_col(df_processed, date_col, date_col)
    
    # 날짜 범위 필터링
    df_filtered_by_date = df_processed[(df_processed[date_col] >= pd.to_datetime(start_date)) & (df_processed[date_col] <= pd.to_datetime(end_date))]
    if df_filtered_by_date.empty: print("No visitor data in the specified date range."); return pd.DataFrame(columns=[date_col]), pd.DataFrame(columns=[date_col])

    # 설악동 데이터
    df_seorakdong_daily = pd.DataFrame(columns=[date_col, 'Total_Visitor_Count']) # 최종 컬럼명 사용
    if 'District_Name' in df_filtered_by_date.columns:
        df_seorakdong_raw = df_filtered_by_date[df_filtered_by_date['District_Name'] == target_district].copy()
        if not df_seorakdong_raw.empty:
            df_seorakdong_daily = df_seorakdong_raw.groupby(date_col, as_index=False)['Total_Visitor_Count_Raw'].sum()
            df_seorakdong_daily.rename(columns={'Total_Visitor_Count_Raw': 'Total_Visitor_Count'}, inplace=True) # 최종 컬럼명
    print(f"Seorakdong visitor data processed. Shape: {df_seorakdong_daily.shape}")
    
    return df_seorakdong_daily

# --- Main Data Processing Pipeline ---
if __name__ == '__main__':
    df_weather = process_weather_data(WEATHER_DATA_DIR, weather_col_map_to_std_korean, 
                                      weather_col_map_korean_to_english, COMMON_DATE_COLUMN, 
                                      TARGET_WEATHER_STATION_NAME)
    
    df_accident = process_accident_data(ACCIDENT_RAW_FILE, accident_col_map_korean_to_english, 
                                        COMMON_DATE_COLUMN, START_DATE_STR, END_DATE_STR)
    
    df_seorakdong_visitor = process_visitor_data(
        VISITOR_RAW_FILE, visitor_col_map_korean_to_english, COMMON_DATE_COLUMN, 
        TARGET_DISTRICT_FOR_VISITORS, START_DATE_STR, END_DATE_STR
    )

    # --- Merge All Processed Data ---
    print("\n--- 4. Merging All Processed Data ---")
    try:
        start_dt_merge = pd.to_datetime(START_DATE_STR)
        end_dt_merge = pd.to_datetime(END_DATE_STR)
        final_df = pd.DataFrame(pd.date_range(start=start_dt_merge, end=end_dt_merge, freq='D'), columns=[COMMON_DATE_COLUMN])
    except Exception as e: print(f"Error creating date range for final merge: {e}"); exit()

    dataframes_to_merge_final = [df_accident, df_weather, df_seorakdong_visitor]
    
    for i, df_merge_component in enumerate(dataframes_to_merge_final):
        if df_merge_component is not None and not df_merge_component.empty and COMMON_DATE_COLUMN in df_merge_component.columns:
            if not pd.api.types.is_datetime64_any_dtype(df_merge_component[COMMON_DATE_COLUMN]):
                df_merge_component[COMMON_DATE_COLUMN] = pd.to_datetime(df_merge_component[COMMON_DATE_COLUMN], errors='coerce')
            final_df = pd.merge(final_df, df_merge_component, on=COMMON_DATE_COLUMN, how='left')
            print(f"  Merged dataframe component {i+1}. Current shape: {final_df.shape}")
        else:
            print(f"  Skipping merge for dataframe component {i+1} (None, empty, or no date column).")

    # --- Final Touches ---
    print("\n--- 5. Applying Final Touches ---")
    for col in final_df.columns: # 결측치 처리
        if col == COMMON_DATE_COLUMN: continue
        if final_df[col].isnull().any():
            if pd.api.types.is_numeric_dtype(final_df[col]):
                final_df[col].fillna(0, inplace=True)
            elif final_df[col].apply(type).eq(list).all(): # 모든 유효값이 리스트인 경우
                final_df[col] = final_df[col].apply(lambda x: x if isinstance(x, list) else [])
            else: # 그 외 (문자열 등)
                final_df[col].fillna("", inplace=True) # 빈 문자열로 채움

    # 날짜 형식 문자열로 변경 (CSV 저장용)
    final_df[COMMON_DATE_COLUMN] = final_df[COMMON_DATE_COLUMN].dt.strftime('%Y-%m-%d')
    
    # (선택 사항) 최종 컬럼 순서 정의
    # desired_order = [COMMON_DATE_COLUMN, 'Total_Rescued_Count', 'Seorakdong_Visitor_Count', ...]
    # final_df = final_df[[col for col in desired_order if col in final_df.columns] + \
    #                     [col for col in final_df.columns if col not in desired_order]]


    # --- Save Final Merged Data ---
    final_df.to_csv(FINAL_OUTPUT_MERGED_FILE, index=False, encoding='utf-8-sig')
    print(f"\n--- Final Merged Data Saved to {FINAL_OUTPUT_MERGED_FILE} ---")
    print(f"Final DataFrame shape: {final_df.shape}")
    print("Final columns:", final_df.columns.tolist())
    print("Sample of final data:")
    print(final_df.head())

    print("\n========== All Data Processing and Merging Complete ==========")


--- 1. Processing Weather Data ---
  Processing weather file: ./weather/4기상청_속초_풍속(2017~2023).csv
  Processing weather file: ./weather/1기상청_속초_기온(2017~2023).csv
  Processing weather file: ./weather/3기상청_속초_습도(2017~2023).csv
  Processing weather file: ./weather/2기상청_속초_강수량(2017~2023).csv
Weather data processed and merged. Shape: (1734, 11)

--- 2. Processing Accident Data ---
Sokcho accident data summarized. Shape: (405, 4)

--- 3. Processing Visitor Data ---
Seorakdong visitor data processed. Shape: (1734, 2)

--- 4. Merging All Processed Data ---
  Merged dataframe component 1. Current shape: (1734, 4)
  Merged dataframe component 2. Current shape: (1734, 14)
  Merged dataframe component 3. Current shape: (1734, 15)

--- 5. Applying Final Touches ---

--- Final Merged Data Saved to final_merged_seorak_data_refactored.csv ---
Final DataFrame shape: (1734, 15)
Final columns: ['Date', 'Total_Rescued_Count', 'Accident_Cause_List', 'Accident_Outcome_List', 'AvgWindSpd(m/s)', 'MaxWindSpd(m